In [186]:
import sys
sys.path.append('../..')
import pandas as pd
import ta
from utilities.data_manager import ExchangeDataManager
from numpy_ext import rolling_apply as rolling_apply_ext

In [11]:
class Strategy():
    def __init__(
        self,
        pair,
        type=["long"],
        params={},
    ):
        self.df_pair = None
        self.df = None
        self.pair = pair
        self.initial_wallet = 1000
        self.use_long = "long" in type
        self.use_short = "short" in type
        self.params = params
        self.result_df = None

    def get_pair_data(self, timeframe, start = 2050, end = 2050):
        exchange = ExchangeDataManager(
            exchange_name=exchange_name,
            path_download="./database/exchanges"
        )

        self.df_pair = exchange.load_data(self.pair, timeframe, start)

    def populate_indicators(self):
        params = self.params
        df = self.df_pair.copy()
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']),
            inplace=True
        )

        # -- Populate indicators --
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
        df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
        df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
        df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)

        df["is_liquidated"] = False
        df["order_open"] = False
        # Trading logic
        order_open = False
        current_order_number = 0
        open_price = 0
        quantity = 0
        trade_result = 0
        # Constants and Initialization
        initial_wallet = self.initial_wallet
        leverage = params["leverage"]  # Fixed leverage
        maintenance_margin_percent = 0.004
        wallet = initial_wallet
        max_equity = initial_wallet  # To track the max equity before a new trade
        max_drawdown = 0

        for i in df.index:
            if df.loc[i, 'open_long_signal'] and not order_open:
                # Open a new order
                current_order_number += 1
                order_open = True
                open_price = df.loc[i, 'open']
                open_wallet = df.loc[i, 'wallet']
                quantity = (wallet / open_price) * leverage
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open

            # Assign order_number to all rows of the current order
            if order_open:
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open
                # Calculate hypothetical_wallet
                hypothetical_wallet = wallet + quantity * (df.loc[i, 'open'] - open_price)
                df.loc[i, 'hypothetical_wallet'] = hypothetical_wallet
                df.loc[i, 'quantity'] = quantity

                # Check for liquidation
                maintenance_margin = (wallet / leverage) * maintenance_margin_percent
                if hypothetical_wallet < maintenance_margin:
                    df.loc[i, 'is_liquidated'] = True
                    df.loc[i, 'trade_result'] = hypothetical_wallet - wallet
                    wallet = 0  # Update wallet with the loss
                    order_open = False  # Close the order

            # Close the order
            if df.loc[i, 'close_long_signal'] and order_open:
                trade_result = quantity * (df.loc[i, 'open'] - open_price)
                trade_result_perc = trade_result / wallet * 100
                wallet += trade_result  # Update wallet with the profit or loss
                order_open = False  # Close the order
                df.loc[i, 'trade_result'] = trade_result
                df.loc[i, 'trade_result_perc'] = trade_result_perc

            # Set wallet to current wallet value
            df.loc[i, 'wallet'] = wallet

        df["drawdown"] = (df["hypothetical_wallet"] - df["wallet"]) / df["wallet"] * 100

        return df


    def get_result_df(self):
        df = self.df
        final_wallet_amount = df.loc[df["order_open"] & df["close_long_signal"], "wallet"].tail(1)
        total_profit = final_wallet_amount  - self.initial_wallet
        total_profit_perc = total_profit / self.initial_wallet * 100
        total_trades = df["order_number"].max()
        avg_trade_profit_perc = df["trade_result_perc"].dropna().mean()
        avg_trade_profit = df["trade_result"].dropna().mean()
        max_drawdown = df["drawdown"].max()

        result_df = pd.DataFrame(
            {
                "params": str(self.params),
                "final_wallet_amount": final_wallet_amount,
                "total_profit": total_profit,
                "total_profit_perc": total_profit_perc,
                "total_trades": total_trades,
                "avg_trade_profit_perc": avg_trade_profit_perc,
                "avg_trade_profit": avg_trade_profit,
                "max_drawdown": max_drawdown,
            }
        )

        self.result_df = result_df

In [28]:
params = {
    "fast_ma": 5,
    "slow_ma": 60,
    "sigma_open": 2.5,
    "sigma_close": 2.7,
    "mean_mrat_lenght": 60,
    "leverage": 2
}

pair = "API3/USDT:USDT"
exchange_name = "binance"
tf = '15m'
oldest_pair = "API3/USDT:USDT"
start_date = "2024-02-01 00:00:00"

In [29]:
strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date)
df1 = strat.populate_indicators()

In [98]:
df1.loc[df1["order_number"] == 8].iloc[[0,-1], :]

,open,high,low,close,volume,fast_ma,slow_ma,mrat,mean_mrat,stdev_mrat,...,close_long_signal,is_liquidated,order_open,wallet,order_number,hypothetical_wallet,quantity,trade_result,trade_result_perc,drawdown
date,,,,,,,,,,,,,,,,,,,,,
2024-03-18 01:15:00,3.3042,3.3230,3.2976,3.3019,94446.6,3.31402,3.344252,0.990960,1.023098,0.012284,...,False,False,True,1322.822932,8.0,1322.822932,800.691806,NaN,NaN,0.000000
2024-03-19 20:15:00,3.1625,3.1695,3.1349,3.1494,114853.5,3.17856,3.010803,1.055718,0.996041,0.046717,...,False,False,True,1322.822932,8.0,1209.364903,800.691806,NaN,NaN,-8.576963


In [27]:
df1.loc[df1["order_open"] & df1["close_long_signal"], "wallet"].tail(5)

date
2024-02-22 19:00:00    38212.813239
2024-02-24 05:15:00    40941.782265
2024-03-02 00:30:00    41963.677759
2024-03-09 04:45:00    51116.104379
2024-03-17 03:30:00    36104.186651
Name: wallet, dtype: float64

In [97]:
df1.tail(10)

,open,high,low,close,volume,fast_ma,slow_ma,mrat,mean_mrat,stdev_mrat,...,close_long_signal,is_liquidated,order_open,wallet,order_number,hypothetical_wallet,quantity,trade_result,trade_result_perc,drawdown
date,,,,,,,,,,,,,,,,,,,,,
2024-03-19 18:00:00,3.1281,3.1284,3.0882,3.1010,98765.7,3.12416,2.980375,1.048244,0.980669,0.039962,...,False,False,True,1322.822932,8.0,1181.821105,800.691806,NaN,NaN,-10.659161
2024-03-19 18:15:00,3.1011,3.1269,3.1011,3.1181,64403.0,3.12208,2.982613,1.046760,0.982114,0.040751,...,False,False,True,1322.822932,8.0,1160.202426,800.691806,NaN,NaN,-12.293445
2024-03-19 18:30:00,3.1181,3.2680,3.1149,3.2479,926916.3,3.14416,2.987477,1.052447,0.983741,0.041569,...,False,False,True,1322.822932,8.0,1173.814187,800.691806,NaN,NaN,-11.264451
2024-03-19 18:45:00,3.2472,3.2530,3.1908,3.1913,262397.7,3.15740,2.990463,1.055823,0.985445,0.042385,...,False,False,True,1322.822932,8.0,1277.183499,800.691806,NaN,NaN,-3.450154
2024-03-19 19:00:00,3.1913,3.2011,3.1646,3.1901,237723.8,3.16968,2.993368,1.058901,0.987191,0.043210,...,False,False,True,1322.822932,8.0,1232.424827,800.691806,NaN,NaN,-6.833727
2024-03-19 19:15:00,3.1901,3.2300,3.1860,3.2163,137732.0,3.19274,2.996615,1.065449,0.989018,0.044153,...,False,False,True,1322.822932,8.0,1231.463997,800.691806,NaN,NaN,-6.906362
2024-03-19 19:30:00,3.2161,3.2161,3.1758,3.1791,98247.2,3.20494,2.999633,1.068444,0.990861,0.045111,...,False,False,True,1322.822932,8.0,1252.281984,800.691806,NaN,NaN,-5.332607
2024-03-19 19:45:00,3.1790,3.1988,3.1711,3.1862,76367.3,3.19260,3.003358,1.063010,0.992587,0.045851,...,False,False,True,1322.822932,8.0,1222.576318,800.691806,NaN,NaN,-7.578234
2024-03-19 20:00:00,3.1859,3.1956,3.1459,3.1618,155783.4,3.18670,3.007012,1.059756,0.994309,0.046398,...,False,False,True,1322.822932,8.0,1228.101091,800.691806,NaN,NaN,-7.160583


In [102]:
df_test = df1.loc[df1["open_long_signal"] | df1["close_long_signal"], ["open_long_signal", "close_long_signal","open", "close"]]

In [103]:
df_test["open_signal_lag"] = df1["open_long_signal"].shift()
df_test["close_signal_lag"] = df1["close_long_signal"].shift()

In [104]:
df_test

,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag
date,,,,,,
2024-02-02 09:45:00,False,True,2.7921,2.8198,False,False
2024-02-02 10:00:00,False,True,2.8204,2.8372,False,True
2024-02-02 10:15:00,False,True,2.8377,2.8857,False,True
2024-02-02 10:30:00,False,True,2.8857,2.9980,False,True
2024-02-02 10:45:00,False,True,2.9978,3.0219,False,True
...,...,...,...,...,...,...
2024-03-17 03:45:00,False,True,3.2634,3.2674,False,True
2024-03-18 01:15:00,True,False,3.3042,3.3019,False,False
2024-03-18 01:30:00,True,False,3.3024,3.3033,True,False


In [105]:
df_filtered  = df_test[
    (~ df_test["open_signal_lag"] & (df_test["open_long_signal"] | df_test["open_long_signal"].isnull())) |
    (~ df_test["close_signal_lag"] & df_test["close_long_signal"])
     ]

In [106]:
df_filtered

,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag
date,,,,,,
2024-02-02 09:45:00,False,True,2.7921,2.8198,False,False
2024-02-04 03:45:00,False,True,3.3686,3.3548,False,False
2024-02-04 16:15:00,True,False,3.1655,3.1642,False,False
2024-02-05 09:45:00,False,True,3.2457,3.2566,False,False
2024-02-06 16:15:00,False,True,2.9936,3.0108,False,False
2024-02-07 07:15:00,False,True,3.1199,3.2122,False,False
2024-02-11 16:00:00,True,False,3.1049,3.1144,False,False
2024-02-12 04:30:00,False,True,3.3268,3.4421,False,False
2024-02-12 07:15:00,False,True,3.7436,3.6922,False,False


In [107]:
df_filtered["open_signal_lag"] = df_filtered["open_long_signal"].shift(fill_value=False)
df_filtered["close_signal_lag"] = df_filtered["close_long_signal"].shift(fill_value=False)

/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/518765445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["open_signal_lag"] = df_filtered["open_long_signal"].shift(fill_value=False)
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/518765445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["close_signal_lag"] = df_filtered["close_long_signal"].shift(fill_value=False)


In [108]:
df_filtered

,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag
date,,,,,,
2024-02-02 09:45:00,False,True,2.7921,2.8198,False,False
2024-02-04 03:45:00,False,True,3.3686,3.3548,False,True
2024-02-04 16:15:00,True,False,3.1655,3.1642,False,True
2024-02-05 09:45:00,False,True,3.2457,3.2566,True,False
2024-02-06 16:15:00,False,True,2.9936,3.0108,False,True
2024-02-07 07:15:00,False,True,3.1199,3.2122,False,True
2024-02-11 16:00:00,True,False,3.1049,3.1144,False,True
2024-02-12 04:30:00,False,True,3.3268,3.4421,True,False
2024-02-12 07:15:00,False,True,3.7436,3.6922,False,True


In [109]:
df_order = df_filtered[
    (df_filtered["open_long_signal"] & (~df_filtered["open_signal_lag"] | df_filtered["open_long_signal"].isnull())) |
    (df_filtered["close_long_signal"] & ~ df_filtered["close_signal_lag"])
     ]

In [110]:
df_order

,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag
date,,,,,,
2024-02-02 09:45:00,False,True,2.7921,2.8198,False,False
2024-02-04 16:15:00,True,False,3.1655,3.1642,False,True
2024-02-05 09:45:00,False,True,3.2457,3.2566,True,False
2024-02-11 16:00:00,True,False,3.1049,3.1144,False,True
2024-02-12 04:30:00,False,True,3.3268,3.4421,True,False
2024-02-20 15:45:00,True,False,3.8491,3.8323,False,True
2024-02-22 19:00:00,False,True,4.1684,4.1259,True,False
2024-02-23 16:15:00,True,False,3.6911,3.7205,False,True
2024-02-24 05:15:00,False,True,3.8229,3.8245,True,False


In [122]:
df_order_2 = df_order.loc[~ ( ~ df_order["close_signal_lag"] & ~ df_order["open_signal_lag"])]

In [123]:
df_order_2

,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag
date,,,,,,
2024-02-04 16:15:00,True,False,3.1655,3.1642,False,True
2024-02-05 09:45:00,False,True,3.2457,3.2566,True,False
2024-02-11 16:00:00,True,False,3.1049,3.1144,False,True
2024-02-12 04:30:00,False,True,3.3268,3.4421,True,False
2024-02-20 15:45:00,True,False,3.8491,3.8323,False,True
2024-02-22 19:00:00,False,True,4.1684,4.1259,True,False
2024-02-23 16:15:00,True,False,3.6911,3.7205,False,True
2024-02-24 05:15:00,False,True,3.8229,3.8245,True,False
2024-02-25 05:15:00,True,False,3.9103,3.9333,False,True


In [124]:
df_order_2["order_number"] = df_order_2["open_long_signal"].cumsum()

/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/1207584165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order_2["order_number"] = df_order_2["open_long_signal"].cumsum()


In [133]:
df_order_2["open_lag"] = df_order_2["open"].shift(-1)

/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/3190006780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order_2["open_lag"] = df_order_2["open"].shift(-1)


In [138]:
leverage = 2
df_order_2

,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag,order_number,wallet,open_lag
date,,,,,,,,,
2024-02-04 16:15:00,True,False,3.1655,3.1642,False,True,1,1000,3.2457
2024-02-05 09:45:00,False,True,3.2457,3.2566,True,False,1,1000,3.1049
2024-02-11 16:00:00,True,False,3.1049,3.1144,False,True,2,1000,3.3268
2024-02-12 04:30:00,False,True,3.3268,3.4421,True,False,2,1000,3.8491
2024-02-20 15:45:00,True,False,3.8491,3.8323,False,True,3,1000,4.1684
2024-02-22 19:00:00,False,True,4.1684,4.1259,True,False,3,1000,3.6911
2024-02-23 16:15:00,True,False,3.6911,3.7205,False,True,4,1000,3.8229
2024-02-24 05:15:00,False,True,3.8229,3.8245,True,False,4,1000,3.9103
2024-02-25 05:15:00,True,False,3.9103,3.9333,False,True,5,1000,3.9591


In [259]:
leverage = 2  # Assuming leverage is defined
wallet = 1000  # Initial wallet balance
quantity = 0  # Initial quantity

# Ensure the DataFrame has 'quantity' and 'trade_result' columns initialized
df_order_2['quantity'] = 0.0
df_order_2['trade_result'] = 0.0

# Iterating over DataFrame rows to process trading signals
for i, row in df_order_2.iterrows():
    # Check if there is a signal to open a long position
    if row['open_long_signal']:
        # Calculate the new quantity based on the current wallet and leverage
        quantity = wallet * leverage / row['open']
        # Update the 'quantity' column with the new quantity
        df_order_2.at[i, 'quantity'] = quantity
        # No change in wallet yet as the position has just opened
        df_order_2.at[i, 'wallet'] = wallet
        # Track the price at which the position was opened
        open = row['open']
    elif row['close_long_signal']:
        # Calculate the trade result based on the difference between current and open price
        trade_result = (row['open'] - open) * quantity
        # Update the 'trade_result' column with the result of the closed trade
        df_order_2.at[i, 'trade_result'] = trade_result
        # Update the wallet with the result of the trade
        wallet += trade_result
        # Reset quantity as the trade is closed
        quantity = 0

    # Update the wallet and quantity for the current row
    df_order_2.at[i, 'wallet'] = wallet
    df_order_2.at[i, 'quantity'] = quantity

# Return the modified DataFrame
df_order_2

/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/934996041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order_2['quantity'] = 0.0
/var/folders/_9/mg2v8fwd04n3k0y4zztq8z7h0000gn/T/ipykernel_55277/934996041.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_order_2['trade_result'] = 0.0


,open_long_signal,close_long_signal,open,close,open_signal_lag,close_signal_lag,order_number,wallet,open_lag,quantity,trade_result
date,,,,,,,,,,,
2024-02-04 16:15:00,True,False,3.1655,3.1642,False,True,1,1000.000000,3.2457,631.811720,0.000000
2024-02-05 09:45:00,False,True,3.2457,3.2566,True,False,1,1050.671300,3.1049,0.000000,50.671300
2024-02-11 16:00:00,True,False,3.1049,3.1144,False,True,2,1050.671300,3.3268,676.782698,0.000000
2024-02-12 04:30:00,False,True,3.3268,3.4421,True,False,2,1200.849381,3.8491,0.000000,150.178081
2024-02-20 15:45:00,True,False,3.8491,3.8323,False,True,3,1200.849381,4.1684,623.963722,0.000000
2024-02-22 19:00:00,False,True,4.1684,4.1259,True,False,3,1400.080997,3.6911,0.000000,199.231616
2024-02-23 16:15:00,True,False,3.6911,3.7205,False,True,4,1400.080997,3.8229,758.625340,0.000000
2024-02-24 05:15:00,False,True,3.8229,3.8245,True,False,4,1500.067817,3.9103,0.000000,99.986820
2024-02-25 05:15:00,True,False,3.9103,3.9333,False,True,5,1500.067817,3.9591,767.239249,0.000000


In [260]:
df1.tail()

,open,high,low,close,volume,fast_ma,slow_ma,mrat,mean_mrat,stdev_mrat,...,close_long_signal,is_liquidated,order_open,wallet,order_number,hypothetical_wallet,quantity,trade_result,trade_result_perc,drawdown
date,,,,,,,,,,,,,,,,,,,,,
2024-03-19 19:15:00,3.1901,3.2300,3.1860,3.2163,137732.0,3.19274,2.996615,1.065449,0.989018,0.044153,...,False,False,True,1322.822932,8.0,1231.463997,800.691806,NaN,NaN,-6.906362
2024-03-19 19:30:00,3.2161,3.2161,3.1758,3.1791,98247.2,3.20494,2.999633,1.068444,0.990861,0.045111,...,False,False,True,1322.822932,8.0,1252.281984,800.691806,NaN,NaN,-5.332607
2024-03-19 19:45:00,3.1790,3.1988,3.1711,3.1862,76367.3,3.19260,3.003358,1.063010,0.992587,0.045851,...,False,False,True,1322.822932,8.0,1222.576318,800.691806,NaN,NaN,-7.578234
2024-03-19 20:00:00,3.1859,3.1956,3.1459,3.1618,155783.4,3.18670,3.007012,1.059756,0.994309,0.046398,...,False,False,True,1322.822932,8.0,1228.101091,800.691806,NaN,NaN,-7.160583
2024-03-19 20:15:00,3.1625,3.1695,3.1349,3.1494,114853.5,3.17856,3.010803,1.055718,0.996041,0.046717,...,False,False,True,1322.822932,8.0,1209.364903,800.691806,NaN,NaN,-8.576963
